## Basic EDA

In [1]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

# our own pipeline
from pipelines.data_prapare import pack_data
# from pipelines.tools import plot_intervals
from pipelines.tools import power_band, one_signal_band_power, power_band_timeslice
from pipelines.data_prapare import read_orginal_data, read_power_band_txt,read_features_table

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# data_source_path = "./data/Tests_EEG_Lintao/"
# filename_list = os.listdir("./data/Tests_EEG_Lintao/")
# path_list = list()
# for item in filename_list:
#     path_list.append(os.path.join(data_source_path, item))

# # alphabet_list, asl_list, alphabet_vision, alphabet_imagination, asl_vision, asl_imagination = pack_data(path_list)
# alphabet_list, asl_list, alphabet_vision, alphabet_imagination, asl_vision, asl_imagination = read_orginal_data(path_list)

# # all labels are same order
# labels = list()
# for item in alphabet_vision:
#     labels.append(item[-1])
    

# bp_data_dict = read_power_band_txt()

In [4]:
aat_vis, aat_img, asl_vis, asl_img = read_features_table()

In [5]:
aat_vis.head()

,label,label_index,ch1_min,ch1_max,ch1_std,ch1_mean,ch1_coefficient_variation,ch1_mean_abs,ch1_AAC,ch1_CARD,...,ch7_coherence,ch8_coherence,ch9_coherence,ch10_coherence,ch11_coherence,ch12_coherence,ch13_coherence,ch14_coherence,ch15_coherence,ch16_coherence
0,I,9,-3411.345591,-3262.952359,31.367440,-3313.606131,-0.009466,3313.606131,6.992184,342.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000
1,F,6,-3454.283292,-3334.187369,22.167027,-3374.728294,-0.006569,3374.728294,6.986747,332.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000
2,Q,17,-3772.192153,-3341.965776,75.384702,-3591.181606,-0.020992,3591.181606,6.906636,413.0,...,0.508136,0.509386,0.660148,0.678768,0.437121,0.62361,0.630649,0.591844,0.544863,0.530359
3,D,4,-3786.139642,-3662.221570,21.228800,-3707.487320,-0.005726,3707.487320,6.090696,334.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000
4,V,22,-3847.428125,-3682.516954,32.712980,-3751.086843,-0.008721,3751.086843,6.442441,347.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000


In [6]:
aat_vis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 832 entries, 0 to 831
Columns: 1122 entries, label to ch16_coherence
dtypes: float64(1120), int64(1), object(1)
memory usage: 7.1+ MB


In [7]:
aat_vis[['label','label_index']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 832 entries, 0 to 831
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   label        832 non-null    object
 1   label_index  832 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 19.5+ KB


So label dtype is object, a lot of processing will skip label

### Check Values

* NAN
* INF

There are a lot ways to process INF in DF, we will use NAN replace INF and give them values like max_column*10

In [8]:
check_nan = aat_vis.isnull().values.any()
print(check_nan)

check_nan = aat_img.isnull().values.any()
print(check_nan)

check_nan = asl_vis.isnull().values.any()
print(check_nan)

check_nan = asl_img.isnull().values.any()
print(check_nan)

False
False
False
False


In [9]:
count_inf = np.isinf(asl_img.iloc[: ,2:]).values.sum()
print(count_inf)

count_inf = np.isinf(asl_vis.iloc[: ,2:]).values.sum()
print(count_inf)

count_inf = np.isinf(aat_img.iloc[: ,2:]).values.sum()
print(count_inf)

count_inf = np.isinf(aat_vis.iloc[: ,2:]).values.sum()
print(count_inf)

13
11
14
11


In our DF, no nan, but have inf

In [10]:
def replace_inf(df, times = 10):
    
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    t = np.array(df.copy())
    for i in range(t.shape[1]):
        temp_col = t[:,i]
        nan_num = np.count_nonzero(temp_col != temp_col) 
        if nan_num != 0: 
            temp_col_not_nan = temp_col[temp_col==temp_col]
            temp_col[np.isnan(temp_col)] = np.max(temp_col_not_nan)*times
    
    return t   

In [11]:
# aat_img_data = replace_inf(aat_img.iloc[: ,2:])

# count_inf = np.isinf(aat_img_data).sum()
# print(count_inf)

A more direct way

In [12]:
inf_times = 2

aat_img_data = aat_img.iloc[: ,2:].copy()
aat_img_data.replace([np.inf, -np.inf], np.nan, inplace=True)
aat_img_data = aat_img_data.fillna(aat_img_data.max()*inf_times)
count_inf = np.isinf(aat_img_data).values.sum()
print(count_inf)

aat_vis_data = aat_vis.iloc[: ,2:].copy()
aat_vis_data.replace([np.inf, -np.inf], np.nan, inplace=True)
aat_vis_data = aat_vis_data.fillna(aat_vis_data.max()*inf_times)
count_inf = np.isinf(aat_vis_data).values.sum()
print(count_inf)

asl_img_data = asl_img.iloc[: ,2:].copy()
asl_img_data.replace([np.inf, -np.inf], np.nan, inplace=True)
asl_img_data = asl_img_data.fillna(asl_img_data.max()*inf_times)
count_inf = np.isinf(asl_img_data).values.sum()
print(count_inf)

asl_vis_data = asl_vis.iloc[: ,2:].copy()
asl_vis_data.replace([np.inf, -np.inf], np.nan, inplace=True)
asl_vis_data = asl_vis_data.fillna(asl_vis_data.max()*inf_times)
count_inf = np.isinf(asl_vis_data).values.sum()
print(count_inf)

0
0
0
0


In [13]:
check_nan = aat_vis_data.isnull().values.any()
print(check_nan)

check_nan = aat_img_data.isnull().values.any()
print(check_nan)

check_nan = asl_vis_data.isnull().values.any()
print(check_nan)

check_nan = asl_img_data.isnull().values.any()
print(check_nan)

False
False
False
False


### Check corr


Because our feature dimension is large and there are many calculation methods, it is inevitable that many features are related to each other.

For machine learning models, some models such as logistic regression are best not to have relevant features(Ok for tree models);

For deep learning models, there is no such problem.

In [14]:
aat_img_corr = aat_img_data.corr()
aat_vis_corr = aat_vis_data.corr()
asl_img_corr = asl_img_data.corr()
asl_vis_corr = asl_vis_data.corr()

In [15]:
# aat_img_corr = aat_img.corr()
# aat_vis_corr = aat_vis.corr()
# asl_img_corr = asl_img.corr()
# asl_vis_corr = asl_vis.corr()

In [16]:
aat_img_corr.head()

,ch1_min,ch1_max,ch1_std,ch1_mean,ch1_coefficient_variation,ch1_mean_abs,ch1_AAC,ch1_CARD,ch1_EMAV,ch1_median,...,ch7_coherence,ch8_coherence,ch9_coherence,ch10_coherence,ch11_coherence,ch12_coherence,ch13_coherence,ch14_coherence,ch15_coherence,ch16_coherence
ch1_min,1.000000,0.999507,-0.148751,0.999689,-0.721245,-0.999689,-0.223806,-0.073495,0.070328,0.999615,...,0.072479,0.076512,0.084021,0.051747,0.063271,0.075972,0.074978,0.079691,0.078802,0.083228
ch1_max,0.999507,1.000000,-0.121846,0.999935,-0.733765,-0.999935,-0.206535,-0.067345,0.079746,0.999937,...,0.070762,0.074774,0.081656,0.049705,0.061441,0.074118,0.073034,0.077841,0.077273,0.080978
ch1_std,-0.148751,-0.121846,1.000000,-0.130183,-0.368268,0.130183,0.411241,0.249739,0.047052,-0.126404,...,-0.070764,-0.062006,-0.084160,-0.071236,-0.078852,-0.071846,-0.081822,-0.058207,-0.053273,-0.073455
ch1_mean,0.999689,0.999935,-0.130183,1.000000,-0.730409,-1.000000,-0.208688,-0.071417,0.084800,0.999986,...,0.072313,0.076298,0.083507,0.051395,0.063139,0.075748,0.074783,0.079432,0.078764,0.082729
ch1_coefficient_variation,-0.721245,-0.733765,-0.368268,-0.730409,1.000000,0.730409,0.083099,-0.059166,-0.152279,-0.732381,...,-0.019872,-0.022705,-0.023370,0.005883,0.003655,-0.016960,-0.010455,-0.018021,-0.030093,-0.027219


In [17]:
aat_img_corr.shape

(1120, 1120)

In [18]:
def corr_features(df_corr, threshold=0.99):
    
    corr_features_list = list()
    for i in range(df_corr.shape[0]):      
        for j in range(0, i):           
            if np.abs(df_corr.iloc[i, j]) >= threshold and i!=j:
                # row and column are same name order
                # print((df_corr.columns[i], df_corr.columns[j]))
                corr_features_list.append((df_corr.columns[i], df_corr.columns[j]))
    
    return set(corr_features_list)

In [19]:
list1 = corr_features(aat_img_data)
list2 = corr_features(aat_vis_data)
list3 = corr_features(asl_img_data)
list4 = corr_features(asl_vis_data)
# list1

In [20]:
len(list1)

245982

In [21]:
intersection_set = list(set(list1).intersection(list2, list3, list4)) 
print(len(intersection_set), len(list1), len(list2), len(list3), len(list4))

227678 245982 245698 247328 247904


In [22]:
intersection_set[15]

('ch15_MAP', 'ch5_mean_energy')

There are a lot of feature pair are high correlated.

We could:

1. keep just one feature in the related features list;
2. just select one feature in the pair in the intersection;
3. just select one feature in the pair for each set.

In [23]:
unuse_feature_list = list()
for item in intersection_set:
    
    strr1 = item[0].replace("ch1_",'').replace("ch2_",'').replace("ch3_",'').replace("ch4_",'').replace("ch5_",'').replace("ch6_",'')
    strr1 = strr1.replace("ch7_",'').replace("ch8_",'').replace("ch9_",'').replace("ch10_",'').replace("ch11_",'').replace("ch12_",'')
    strr1 = strr1.replace("ch13_",'').replace("ch14_",'').replace("ch15_",'').replace("ch16_",'')
    
    strr2 = item[1].replace("ch1_",'').replace("ch2_",'').replace("ch3_",'').replace("ch4_",'').replace("ch5_",'').replace("ch6_",'')
    strr2 = strr2.replace("ch7_",'').replace("ch8_",'').replace("ch9_",'').replace("ch10_",'').replace("ch11_",'').replace("ch12_",'')
    strr2 = strr2.replace("ch13_",'').replace("ch14_",'').replace("ch15_",'').replace("ch16_",'')
    
    if strr1 != strr2 and ((strr1, strr2) not in unuse_feature_list) and ((strr2, strr1) not in unuse_feature_list):
        unuse_feature_list.append((strr1, strr2))

In [24]:
len(unuse_feature_list)

1604

In [25]:
unuse_feature_list[0]

('mean_target_energy', 'shannon_entropy')

In [26]:
corr_list = [[unuse_feature_list[0][0], unuse_feature_list[0][1]]]

for item in unuse_feature_list:
    
    for list_item in corr_list:

        if (item[0] in list_item) and (item[1] not in list_item):
            list_item.append(item[1])
        elif (item[1] in list_item) and (item[0] not in list_item):
            list_item.append(item[0])
        elif (item[0] not in list_item) and (item[1] not in list_item):           
            sub_list = [item[0], item[1]]
            corr_list.append(sub_list)
        elif (item[0] in list_item) and (item[1] in list_item):
            break
        break
        
print('Finish building list!')
results = [set(corr_list[0])]
for i in range(1, len(corr_list)):
    if set(corr_list[i]).issubset(set(corr_list[0])):
        continue
    else:
        results.append(set(corr_list[i]))

Finish building list!


In [27]:
len(results)

1

In [28]:
# results[0]

We can just keep one feature in those lists, but after careful observation, we found that the power band features are also related.

So in the end, don't delete the feature directly, but delete it in the original feature pair list, but this process will be a little more complicated.

## Comparison 

* Percent subtraction
* Distance
* Correlation coefficient


### Percent subtraction

An interesting discovery, if we write code like above, there will be only 0% and 100%, which is caused by the speciality of the round function

In [29]:
def percent_subtraction(alist1, alist2):
    
    differences = list()
    for i in range(len(alist1)):
        
        diff = np.abs(alist1[i] - alist2[i])
        
        max1 = diff*100/np.abs(alist1[i])
        max2 = diff*100/np.abs(alist2[i])
        diff = round(max(max1, max2))   #  *100
        
        if diff >= 95:
            differences.append("100%")
        elif 95> diff >=85:
            differences.append("90%")
        elif 85> diff >=75:
            differences.append("80%")
        elif 75> diff >=65:
            differences.append("70%") 
        elif 65> diff >=55:
            differences.append("60%")
        elif 55> diff >=45:
            differences.append("50%")
        elif 45> diff >=35:
            differences.append("40%")
        elif 35> diff >=25:
            differences.append("30%")
        elif 25> diff >=15:
            differences.append("20%")
        elif 15> diff >=5:
            differences.append("10%")
        else:
            differences.append("0%")
    return differences

Mean big class

In [30]:
aat_img_mean = np.mean(np.array(aat_img_data), axis=0, keepdims=False)
# aat_img_mean.shape

aat_vis_mean = np.mean(np.array(aat_vis_data), axis=0, keepdims=False)
asl_img_mean = np.mean(np.array(asl_img_data), axis=0, keepdims=False)
asl_vis_mean = np.mean(np.array(asl_vis_data), axis=0, keepdims=False)

Compare vision and imagination for alphabet/sign

In [31]:
# Counter(percent_subtraction(aat_img_mean, aat_vis_mean))

In [32]:
# Counter(percent_subtraction(asl_img_mean, asl_vis_mean))

In [33]:
# Counter(percent_subtraction(aat_img_mean, asl_img_mean))

In [34]:
# Counter(percent_subtraction(aat_vis_mean, asl_vis_mean))

So in theory, it is completely possible to complete 26 classifications.

In addition, the results also give us another way of feature selection. If the model cannot select features well by itself, we can consider using this method to select features for machine learning model training.


In [35]:
# concat dict

dict1 = dict(Counter(percent_subtraction(aat_img_mean, aat_vis_mean)))
dict2 = dict(Counter(percent_subtraction(asl_img_mean, asl_vis_mean)))
dict3 = dict(Counter(percent_subtraction(aat_img_mean, asl_img_mean)))
dict4 = dict(Counter(percent_subtraction(aat_vis_mean, asl_vis_mean)))

com_dict = {key:[dict1[key], dict2[key], dict3[key],dict4[key]] for key in dict1.keys()}
com_df = pd.DataFrame(com_dict, index=['aat_img VS aat_vis', 'asl_img VS asl_vis','aat_img VS asl_img','aat_vis VS asl_vis'], 
                      columns=['0%', '10%','20%','30%','40%','50%','60%','70%','80%','90%','100%'])
com_df

,0%,10%,20%,30%,40%,50%,60%,70%,80%,90%,100%
aat_img VS aat_vis,740,146,61,29,17,19,11,4,7,6,80
asl_img VS asl_vis,778,137,50,23,11,13,9,5,3,6,85
aat_img VS asl_img,306,198,86,57,30,50,46,35,26,33,253
aat_vis VS asl_vis,301,193,90,63,32,50,40,33,17,36,265


Mean alphabet vision W V A:

Why W V A, because we compared MNE image for those three in the previous work.

And we just want to make sure the 26 classification is possible.

In [36]:
W_mean_aat_vis = np.mean(np.array(aat_vis_data[aat_vis['label'] == 'W']), axis=0, keepdims=False)
V_mean_aat_vis = np.mean(np.array(aat_vis_data[aat_vis['label'] == 'V']), axis=0, keepdims=False)
A_mean_aat_vis = np.mean(np.array(aat_vis_data[aat_vis['label'] == 'A']), axis=0, keepdims=False)

W_mean_aat_img = np.mean(np.array(aat_img_data[aat_img['label'] == 'W']), axis=0, keepdims=False)
V_mean_aat_img = np.mean(np.array(aat_img_data[aat_img['label'] == 'V']), axis=0, keepdims=False)
A_mean_aat_img = np.mean(np.array(aat_img_data[aat_img['label'] == 'A']), axis=0, keepdims=False)

W_mean_asl_vis = np.mean(np.array(asl_vis_data[asl_vis['label'] == 'W']), axis=0, keepdims=False)
V_mean_asl_vis = np.mean(np.array(asl_vis_data[asl_vis['label'] == 'V']), axis=0, keepdims=False)
A_mean_asl_vis = np.mean(np.array(asl_vis_data[asl_vis['label'] == 'A']), axis=0, keepdims=False)

W_mean_asl_img = np.mean(np.array(asl_img_data[asl_img['label'] == 'W']), axis=0, keepdims=False)
V_mean_asl_img = np.mean(np.array(asl_img_data[asl_img['label'] == 'V']), axis=0, keepdims=False)
A_mean_asl_img = np.mean(np.array(asl_img_data[asl_img['label'] == 'A']), axis=0, keepdims=False)

In [37]:
dict1 = dict(Counter(percent_subtraction(W_mean_aat_vis, V_mean_aat_vis)))
dict2 = dict(Counter(percent_subtraction(W_mean_aat_vis, A_mean_aat_vis)))
dict3 = dict(Counter(percent_subtraction(A_mean_aat_vis, V_mean_aat_vis)))
dict4 = dict(Counter(percent_subtraction(W_mean_aat_img, V_mean_aat_img)))
dict5 = dict(Counter(percent_subtraction(W_mean_aat_img, A_mean_aat_img)))
dict6 = dict(Counter(percent_subtraction(A_mean_aat_img, V_mean_aat_img)))
dict7 = dict(Counter(percent_subtraction(W_mean_asl_vis, V_mean_asl_vis)))
dict8 = dict(Counter(percent_subtraction(W_mean_asl_vis, A_mean_asl_vis)))
dict9 = dict(Counter(percent_subtraction(A_mean_asl_vis, V_mean_asl_vis)))
dict10 = dict(Counter(percent_subtraction(W_mean_asl_img, V_mean_asl_img)))
dict11 = dict(Counter(percent_subtraction(W_mean_asl_img, A_mean_asl_img)))
dict12 = dict(Counter(percent_subtraction(A_mean_asl_img, V_mean_asl_img)))

com_dict = {key:[dict1[key], dict2[key], dict3[key],dict4[key],dict5[key],dict6[key],dict7[key],dict8[key],
                 dict9[key],dict10[key],dict11[key],dict12[key]] for key in dict1.keys()}
com_df = pd.DataFrame(com_dict, index=['aat_vis: W V', 'aat_vis: W A','aat_vis: V A','aat_img: W V', 'aat_img: W A','aat_img: V A',
                                      'asl_vis: W V', 'asl_vis: W A','asl_vis: V A','asl_img: W V', 'asl_img: W A','asl_img: V A',], 
                      columns=['0%', '10%','20%','30%','40%','50%','60%','70%','80%','90%','100%'])
com_df

,0%,10%,20%,30%,40%,50%,60%,70%,80%,90%,100%
aat_vis: W V,668,199,61,27,11,17,6,9,7,5,110
aat_vis: W A,612,247,58,33,10,13,9,19,3,3,113
aat_vis: V A,688,193,50,26,18,12,4,6,9,2,112
aat_img: W V,648,197,72,37,16,13,11,15,5,9,97
aat_img: W A,582,265,60,25,30,11,8,9,8,6,116
aat_img: V A,660,196,54,30,21,14,6,8,4,6,121
asl_vis: W V,620,183,74,22,23,11,15,14,9,8,141
asl_vis: W A,615,210,59,24,23,18,15,8,9,4,135
asl_vis: V A,708,149,58,41,18,12,7,4,6,2,115
asl_img: W V,608,185,79,21,22,19,13,11,4,1,157


Rewrite the function, let's the top 5 or top 10 different features


In [38]:
def percent_subtraction(alist1, alist2, feature_name_list,top_diff='all'):
    """
    top_diff: could be str - all or a number
    """
    differences = list()
    top_diffs = dict()
    for i in range(len(alist1)):
        
        diff = np.abs(alist1[i] - alist2[i])
        
        max1 = diff*100/np.abs(alist1[i])
        max2 = diff*100/np.abs(alist2[i])
        diff = round(max(max1, max2))   #  *100
        
        if diff >= 95:
            differences.append("100%")
            top_diffs[feature_name_list[i]] = diff 
        elif 95> diff >=85:
            differences.append("90%")
        elif 85> diff >=75:
            differences.append("80%")
        elif 75> diff >=65:
            differences.append("70%") 
        elif 65> diff >=55:
            differences.append("60%")
        elif 55> diff >=45:
            differences.append("50%")
        elif 45> diff >=35:
            differences.append("40%")
        elif 35> diff >=25:
            differences.append("30%")
        elif 25> diff >=15:
            differences.append("20%")
        elif 15> diff >=5:
            differences.append("10%")
        else:
            differences.append("0%")
    
    # print(top_diffs.items())
    tt = sorted(top_diffs.items(), key=lambda d: d[1], reverse=True)
    if top_diff == 'all': 
        return differences, tt
    elif isinstance(top_diff,int):
        return differences, tt[0:top_diff]
    else:
        raise ValueError

In [39]:
# # differences, tt = percent_subtraction(aat_img_mean, aat_vis_mean, np.array(aat_img.columns), 'all')
# differences, tt = percent_subtraction(aat_img_mean, aat_vis_mean, np.array(aat_img.columns), 10)
# tt

In [40]:
# differences, tt = percent_subtraction(asl_img_mean, asl_vis_mean, np.array(aat_img.columns), 10)
# tt

In [41]:
# differences, tt = percent_subtraction(aat_img_mean, asl_img_mean, np.array(aat_img.columns), 10)
# tt

In [42]:
# differences, tt = percent_subtraction(aat_vis_mean, asl_vis_mean, np.array(aat_img.columns), 10)
# tt

In [43]:
# differences, tt = percent_subtraction(W_mean_aat_vis, V_mean_aat_vis, np.array(aat_img.columns), 10)
# tt

In [44]:
# differences, tt = percent_subtraction(A_mean_aat_vis, V_mean_aat_vis, np.array(aat_img.columns), 10)
# tt

In [45]:
# differences, tt = percent_subtraction(W_mean_aat_vis, V_mean_aat_vis, np.array(aat_img.columns), 10)
# tt

After executing the code commented out above, we can find that the features that appear more frequently are(>2):

(There is another point to note. We have obtained the correlation of features in the previous section. We can find that some of the following features are also related to each other.)

[feature name, appearance times]
* std var 16
* peak 12
* svd_entropy 7
* lyapunov_exponent 5
* hjorth_mobility 5
* LZC 5
* rms 5

* pb_beta 2
* pb_theta 2
* pb_delta 2
* 2_max_diff  3
* pb_gamma 3
* LRSSV 2


In addition, the statistical characteristics of std, and the number of peaks seem to have a large influence here, and we can reconfirm the importance of these characteristics in the model section later.

Finally, we can see the existence of the power band, which is not difficult to understand why the power band is the most classic and most useful feature.

### Distance

When comparing features by distance, it is best to normalize the features to prevent numerical interference.

In addition, considering the dimension of the feature, the calculation of some distances is not suitable for our situation.

And this is just a basic observation, see some examples (W V A) of our signals, if want to get some conclusions, it's better to do this process for all labels(26 labels).

In [46]:
from scipy.spatial import distance
from sklearn.preprocessing import MinMaxScaler
# [num_samples, num_features]

Just compare the example letters

In [47]:
mm = MinMaxScaler()
mm_data = mm.fit_transform(np.array(aat_vis_data[aat_vis['label'] == 'W']))
W_mean_aat_vis = np.mean(mm_data, axis=0, keepdims=False)

mm = MinMaxScaler()
mm_data = mm.fit_transform(np.array(aat_vis_data[aat_vis['label'] == 'V']))
V_mean_aat_vis = np.mean(mm_data, axis=0, keepdims=False)

mm = MinMaxScaler()
mm_data = mm.fit_transform(np.array(aat_vis_data[aat_vis['label'] == 'A']))
A_mean_aat_vis = np.mean(mm_data, axis=0, keepdims=False)

#### Minkowski Distance

Minkowski distance is typically used with param p being 1 or 2, which correspond to the Manhattan distance and the Euclidean distance, respectively. In the limiting case of p reaching infinity, we obtain the Chebyshev distance.

(p < 1 may be work for high dimension features)

In [48]:
# similarity

p_distance = 1
print("The Manhattan Distance between W V is::",distance.minkowski(W_mean_aat_vis, V_mean_aat_vis, p_distance))
print("The Manhattan Distance between W A is::",distance.minkowski(W_mean_aat_vis, A_mean_aat_vis, p_distance))
print("The Manhattan Distance between A V is::",distance.minkowski(A_mean_aat_vis, V_mean_aat_vis, p_distance))

print()
p_distance = 2
print("The Euclidean Distance between W V is::",distance.minkowski(W_mean_aat_vis, V_mean_aat_vis, p_distance))
print("The Euclidean Distance between W A is::",distance.minkowski(W_mean_aat_vis, A_mean_aat_vis, p_distance))
print("The Euclidean Distance between A V is::",distance.minkowski(A_mean_aat_vis, V_mean_aat_vis, p_distance))

The Manhattan Distance between W V is:: 73.28459168324623
The Manhattan Distance between W A is:: 70.12956041931734
The Manhattan Distance between A V is:: 77.48407348500993

The Euclidean Distance between W V is:: 3.3860587239875986
The Euclidean Distance between W A is:: 3.081774937791531
The Euclidean Distance between A V is:: 3.4698322071738823


In [49]:
# difference
differences, tt = percent_subtraction(W_mean_aat_vis, V_mean_aat_vis, np.array(aat_img.columns))
tt_keys1 =  [item[0] for item in tt ]

differences, tt = percent_subtraction(A_mean_aat_vis, V_mean_aat_vis, np.array(aat_img.columns))
tt_keys2 =  [item[0] for item in tt ]

differences, tt = percent_subtraction(W_mean_aat_vis, A_mean_aat_vis, np.array(aat_img.columns))
tt_keys3 =  [item[0] for item in tt ]

intersection_set = list(set(tt_keys1).intersection(tt_keys2, tt_keys3)) 

mm = MinMaxScaler()
mm_data = mm.fit_transform(np.array(aat_vis_data[aat_vis['label'] == 'W'][intersection_set]))
W_mean_aat_vis_dif = np.mean(mm_data, axis=0, keepdims=False)

mm = MinMaxScaler()
mm_data = mm.fit_transform(np.array(aat_vis_data[aat_vis['label'] == 'V'][intersection_set]))
V_mean_aat_vis_dif = np.mean(mm_data, axis=0, keepdims=False)

mm = MinMaxScaler()
mm_data = mm.fit_transform(np.array(aat_vis_data[aat_vis['label'] == 'A'][intersection_set]))
A_mean_aat_vis_dif = np.mean(mm_data, axis=0, keepdims=False)

# get the most different features and calculate their differences distance

p_distance = 1
print("The Manhattan Distance between W V is::",distance.minkowski(W_mean_aat_vis_dif, V_mean_aat_vis_dif, p_distance))
print("The Manhattan Distance between W A is::",distance.minkowski(W_mean_aat_vis_dif, A_mean_aat_vis_dif, p_distance))
print("The Manhattan Distance between A V is::",distance.minkowski(A_mean_aat_vis_dif, V_mean_aat_vis_dif, p_distance))

print()
p_distance = 2
print("The Euclidean Distance between W V is::",distance.minkowski(W_mean_aat_vis_dif, V_mean_aat_vis_dif, p_distance))
print("The Euclidean Distance between W A is::",distance.minkowski(W_mean_aat_vis_dif, A_mean_aat_vis_dif, p_distance))
print("The Euclidean Distance between A V is::",distance.minkowski(A_mean_aat_vis_dif, V_mean_aat_vis_dif, p_distance))

The Manhattan Distance between W V is:: 0.3251827253844168
The Manhattan Distance between W A is:: 0.18830570326685012
The Manhattan Distance between A V is:: 0.13687702211756667

The Euclidean Distance between W V is:: 0.17425313603212692
The Euclidean Distance between W A is:: 0.10245725459492828
The Euclidean Distance between A V is:: 0.07263772004149686


But what we can find is that the average distance between vectors is not as large as we thought, so only some features are important factors for classification.

#### Cosine distance and cosine similarity

cosine distance = 1 - cosine similarity

In [50]:
# similarity distance
print("The Cosine Distance between W V is::",distance.cosine(W_mean_aat_vis, V_mean_aat_vis))
print("The Cosine Distance between W A is::",distance.cosine(W_mean_aat_vis, A_mean_aat_vis))
print("The Cosine Distance between A V is::",distance.cosine(A_mean_aat_vis, V_mean_aat_vis))

print()
# difference distance
print("The Cosine Distance between W V is::",distance.cosine(W_mean_aat_vis_dif, V_mean_aat_vis_dif))
print("The Cosine Distance between W A is::",distance.cosine(W_mean_aat_vis_dif, A_mean_aat_vis_dif))
print("The Cosine Distance between A V is::",distance.cosine(A_mean_aat_vis_dif, V_mean_aat_vis_dif))

print()
# similarity distance
print("The Cosine Similarity between W V is::",1 - distance.cosine(W_mean_aat_vis, V_mean_aat_vis))
print("The Cosine Similarity between W A is::",1 - distance.cosine(W_mean_aat_vis, A_mean_aat_vis))
print("The Cosine Similarity between A V is::",1 - distance.cosine(A_mean_aat_vis, V_mean_aat_vis))

The Cosine Distance between W V is:: 0.0245760224914513
The Cosine Distance between W A is:: 0.02090346621240824
The Cosine Distance between A V is:: 0.025811939904214154

The Cosine Distance between W V is:: 0.009727144367590035
The Cosine Distance between W A is:: 0.00410134449095223
The Cosine Distance between A V is:: 0.0013194780840022347

The Cosine Similarity between W V is:: 0.9754239775085487
The Cosine Similarity between W A is:: 0.9790965337875918
The Cosine Similarity between A V is:: 0.9741880600957858


The values of cosine distance and similarity look even more indistinguishable.



### Correlation coefficient

* Pearson Correlation Coefficient
* Spearman Correlation

PS: Jaccard/Tanimoto similarity coefficient basically used for binary vectors.

Pearson's correlation coefficient and p-value were used to test for non-correlation.

The Pearson correlation coefficient measures the linear relationship between two datasets. The calculation of the p-value relies on the assumption that each dataset is normally distributed. Like other correlation coefficients, this Pearson correlation coefficient takes a value between -1 and +1, with 0 indicating no correlation. A correlation of -1 or +1 indicates that there is a clear linear relationship. A positive correlation means that as x increases, y also increases. A negative correlation means that as x increases, y decreases.

The p-value roughly represents the probability that uncorrelated system-generated datasets have a Pearson correlation at least as extreme as the data computed from those datasets.

The p-value roughly represents the probability that the uncorrelated system produces datasets that are at least as extreme as the datasets computed from them.

The p-value represents a test of the null hypothesis that x and y are uncorrelated (ie, the true population correlation coefficient is zero). So sample correlation coefficients close to zero (i.e. weak correlations) will tend to give you larger p-values, while coefficients close to 1 or -1 (i.e. strong positive/negative correlations) will give you smaller p-values.

In [51]:
from scipy.stats import pearsonr, spearmanr

So in fact, the correlation coefficient method used to compare features will not directly see the similarity and difference like our previous calculations.

In [52]:
r, p_value = pearsonr(W_mean_aat_vis, V_mean_aat_vis)
print("correlation coefficient", r, "\t p value", p_value)

correlation coefficient 0.8392653232272731 	 p value 4.027412532438404e-298


In [53]:
r, p_value = pearsonr(W_mean_aat_vis, A_mean_aat_vis)
print("correlation coefficient", r, "\t p value", p_value)

correlation coefficient 0.8764135358747893 	 p value 0.0


In [54]:
r, p_value = pearsonr(A_mean_aat_vis, V_mean_aat_vis)
print("correlation coefficient", r, "\t p value", p_value)

correlation coefficient 0.845439017148939 	 p value 8.07131563634394e-307


spearmanr

In [55]:
r, p_value = spearmanr(A_mean_aat_vis, V_mean_aat_vis)
print("correlation coefficient", r, "\t p value", p_value)

correlation coefficient 0.8427478663390278 	 p value 5.554994252971267e-303


So the mean features values of these example letters are strongly correlated with very small p-values.

This does not seem to explain much, but the features that lead to irrelevance are likely to be our classification decision features.

## Feature selection

When the feature dimension is large, there may be a curse of dimensionality.

Therefore, we need to select appropriate features to train the model. We have the following selection methods (we prefer to let the model choose by itself, so many of the methods here will only perform a simple test, only when the model cannot select features well , that is, when the model performance is not good, we will return to this part)

Some methods that we could use:

* Irrelevant features(between target and features)
* Low variance features
* P value
* Characteristic/beta coefficient
* High correlation features(multicollinearity)
* No difference features
* Variance Inflation Factor (VIF)
* Selection based on feature importance
* Automatic feature selection with Scikit Learn(i.e. L1 Regularization)
* PCA, LDA


### Some methods and codes
P value and Characteristic/beta coefficient, they are usually suitable for regression models.

#### Irrelevant features

In [56]:
# # drop uncorrelated numeric features (threshold <0.2)
# corr = abs(df.corr().loc['price'])
# corr = corr[corr<0.2]
# cols_to_drop = corr.index.to_list()
# df = df.drop(cols_to_drop, axis=1)

#### Low variance features

In [57]:
# variance of numeric features
# (df.select_dtypes(include=np.number).var().astype('str'))

# from sklearn.feature_selection import VarianceThreshold
# fit_transform

#### VIF

Variance inflation factor (VIF) is another measure of multicollinearity. It is measured as the ratio of the overall model variance to the variance of each independent feature. A high VIF for a feature indicates that it is related to one or more other features. Based on experience:

* VIF = 1 means no correlation
* VIF = 1-5 Moderate correlation
* VIF >5 high correlation

VIF is a useful technique to eliminate multicollinearity features

In [58]:
# from statsmodels.stats.outliers_influence import variance_inflation_factor

# # calculate VIF
# vif = pd.Series([variance_inflation_factor(X.values, i) for i in range(X.shape[1])], index=X.columns)

# # display VIFs in a table
# index = X_train.columns.tolist()
# vif_df = pd.DataFrame(vif, index = index, columns = ['vif']).sort_values(by = 'vif', ascending=False)
# vif_df[vif_df['vif']<10]

### Main methods for our case

High correlation features(multicollinearity) and No difference features, we have done in the previous section;

#### Selection based on feature importance

Decision trees/random forests split the data using a feature that minimizes impurity (measured as Gini Coefficient/gain or information gain). Finding the best features is a critical part of how algorithms work in classification tasks. We can access the best features through the feature_importances_ attribute.

In [59]:
# from sklearn.ensemble import RandomForestClassifier

# # instantiate model
# model = RandomForestClassifier(n_estimators=200, random_state=0)
# # fit model
# model.fit(X,y)


# # feature importance
# importances = model.feature_importances_

# # visualization
# cols = X.columns(pd.DataFrame(importances, cols, columns = ['importance'])
#                  .sort_values(by='importance', ascending=True)
#                  .plot(kind='barh', figsize=(4,10)))

The output above shows the importance of each feature in reducing each node/split.

Since the random forest classifier has many estimators (such as 200 decision trees in the example above), confidence intervals can be used to calculate estimates of relative importance.

In [60]:
# # calculate standard deviation of feature importances
# std = np.std([i.feature_importances_ for i in model.estimators_], axis=0)

# # visualization
# feat_with_importance = pd.Series(importances, X.columns)
# fig, ax = plt.subplots(figsize=(12,5))
# feat_with_importance.plot.bar(yerr=std, ax=ax)
# ax.set_title("Feature importances")
# ax.set_ylabel("Mean decrease in impurity")

#### Automatic feature selection with Scikit Learn

There is a complete module in the sklearn library that handles feature selection in just a few lines of code.

There are many automated processes in sklearn, but here I will show only a few:

In [61]:
# from sklearn.feature_selection import SelectKBest, chi2, SelectPercentile, SelectFromModel, 
# from sklearn.feature_selection import SequentialFeatureSelector, SequentialFeatureSelector

##### Chi-square based techniques

Chi-square based techniques select a certain number of user-defined features (k) based on some predefined scores. These scores are determined by calculating the chi-square statistic between the x (independent) and y (dependent) variables. In sklearn, all that needs to be done is to decide how many features to keep.

In [62]:
# # select K best features
# X_best = SelectKBest(chi2, k=10).fit_transform(X,y)

# # keep 75% top features
# X_best = SelectPercentile(chi2, percentile = 75).fit_transform(X,y)

# # number of best features
# X_best.shape[1]

##### Regularization

Regularization reduces overfitting. If you have too many features, regularization controls their effect, either by shrinking the feature coefficients (called L2 regularization) or setting some of the feature coefficients to zero (called L1 regularization).

Some models have built-in L1/L2 regularization as hyperparameters to penalize features. These functions can be eliminated using the converter SelectFromModel.

In [63]:
# from sklearn.svm import LinearSVC

# model = LinearSVC(penalty= 'l1', C = 0.002, dual=False)
# model.fit(X,y)

# # select features using the meta transformer
# selector = SelectFromModel(estimator = model, prefit=True)

# X_new = selector.transform(X)
# X_new.shape[1]


# # names of selected features
# feature_names = np.array(X.columns)
# feature_names[selector.get_support()]

##### sequential method

The sequential method is a classic statistical technique. In this case add/remove one feature at a time and check the model performance until it is optimized for the needs.

There are two variants of the sequential method. Forward selection techniques start with 0 features, then add a feature that minimizes errors; then add another feature, and so on.

Backward selection works in the opposite direction. The model starts with all the included features and computes the error; it then removes one feature that can further reduce the error. Repeat this process until the desired number of features remain.

In [64]:
# # instantiate model
# model = RandomForestClassifier(n_estimators=100, random_state=0)

# # select features
# selector = SequentialFeatureSelector(estimator=model, n_features_to_select=10, direction='backward', cv=2)
# selector.fit_transform(X,y)

# # check names of features selected
# feature_names = np.array(X.columns)
# feature_names[selector.get_support()]

#### PCA

The main purpose of PCA is to reduce the dimensionality of high-dimensional feature spaces. The original features are reprojected to new dimensions (i.e. principal components). The ultimate goal is to find the number of features that best explains the variance in the data.

In [65]:
# from sklearn.decomposition import PCA

# # scaling data
# X_scaled = scaler.fit_transform(X)

# # fit PCA to data
# pca = PCA()
# pca.fit(X_scaled)
# evr = pca.explained_variance_ratio_

# # visualizing the variance explained by each principal components
# plt.figure(figsize=(12, 5))
# plt.plot(range(0, len(evr)), evr.cumsum(), marker="o", linestyle="--")
# plt.xlabel("Number of components")
# plt.ylabel("Cumulative explained variance")

#### LDA

In general, LDA(Linear Discriminant Analysis) is very similar to Principal Component Analysis (PCA), but unlike PCA, which finds the axis components that maximize the variance of all data, LDA is concerned with the axis components that maximize the degree of discrimination between classes.


In a nutshell, LDA projects the feature space (multi-dimensional samples in a dataset) into a k-dimensional subspace with smaller dimensions (k≤n−1), while maintaining the information that distinguishes classes. In general, dimensionality reduction not only reduces the computational cost of classification tasks, but also reduces the error of parameter estimation, thereby avoiding overfitting.

<b>PCA vs. LDA</b>

Linear discriminant analysis (LDA) and principal component analysis (PCA) are both commonly used linear transformation dimensionality reduction methods. PCA is an "unsupervised" algorithm that doesn't care about class labels, but instead focuses on finding those directions in the dataset that maximize variance (aka "principal components"); LDA is "supervised", which computes is another class-specific direction (or linear discriminator "device") - these directions characterize the axes that maximize the degree of discrimination between classes.

Although it sounds intuitively that LDA is better than PCA for multi-classification tasks when class information is known, this is not necessarily the case.

For example, comparing the classification accuracy of image recognition tasks after PCA or LDA processing, if the number of samples is relatively small, PCA is better than LDA (PCA vs. LDA, A.M. Martinez et al., 2001). It is not uncommon to use LDA and PCA in combination. For example, PCA is used first and then LDA is used for dimensionality reduction.

It should be pointed out that LDA assumes that the data obeys a normal distribution, that different features are statistically independent of each other, and that the covariance matrices of various types of data are equal. However, this only refers to the use of LDA as a classifier. When LDA is used for dimensionality reduction, even if the data does not meet these assumptions, LDA can usually achieve good results.

In [66]:
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# lda = LinearDiscriminantAnalysis(n_components=2)
# lda.fit(X,y)
# X_new = lda.transform(X)